In [1]:
def dimac_StartsWith_p(Input_Lines, name, info):
        Info = Input_Lines.pop(0)
        
        Input_Lines = [x[0:-1] for x in Input_Lines]
        
        if info:
                Number_of_Variables = Info[-2]
                Number_of_Clauses   = Info[-1]
                
                print(f"Number of Variables present in {name} : ", Number_of_Variables)
                print(f"Number of Clauses present   in {name}  : ", Number_of_Clauses)
                print(f"The first 5 clauses in {name} file :", Input_Lines[0:5])
                
                return Input_Lines 
        else:

                return Input_Lines    
    

def dimac_Reader(input_file, name, info = False, sudoku_vals = False): #DIMACS Reader; returns lists of lists
    Input_Lines = []
    if sudoku_vals == True:
            Input_Lines= [line.strip('\n').strip('0') for line in input_file.readlines()]
                        
           # for i in Input_Lines:
           #     print(i)
           #     i = i.strip('\n').strip('0')
           #     print(i)
            return Input_Lines
    else:   
        for line in input_file.readlines():
                Input_Lines.append(line.strip("\n").split(" "))
        
        #Start of information extraction
        if Input_Lines[0][0] == 'c':
                Input_Lines =  Input_Lines[1:]
                return dimac_StartsWith_p(Input_Lines, name, info)
            
        elif Input_Lines[0][0] == 'p':        
                return dimac_StartsWith_p(Input_Lines, name,info)
        
        else:
                Input_Lines = [x[0:-1] for x in Input_Lines]  
                if info:
                        print(f"Number of Clauses present in {name} file  : ", len(Input_Lines))
                        print(f"The first 5 clauses in {name} file :", Input_Lines[0:5])
                        
                        return Input_Lines
                else:
                        return Input_Lines
            
        #End of information extraction

In [2]:
def get_literals():
    literals = []
    for i in range (111, 1000):
        if ('0' not in str(i)):
            literals.append(str(i))
    print(len(literals))
    return literals

In [3]:
def join_input(sudoku_rules, input_sudoku, merge = True):
    "Takes in the rules and the input, cleans and merges them"
    "Input: rules and input_sudoku"
    "Output: vars assigned and merged rules and sudoku"
  #Check which clauses are True based on values which are assigned True of False
    assigned = []
    #print(len(sudoku_rules))
    #print(len( input_sudoku))
    for item in input_sudoku:
        #print(item)
        if (type(item)!= int): item = int(item)
        assert (type(item == int))
        for rule in sudoku_rules:
            #print(rule)
            #If input assigns var in clause True,then clause is True
            if item in rule:
                #print('remove rule ', rule)
                sudoku_rules.remove(rule)
                assigned.append(item)
            elif -item in rule:
                rule.remove(-item)
                #print('remove -item ', item )
    assigned = list(set(assigned))
    #print(len(sudoku_rules))
    sudoku_rules = list(filter(None, sudoku_rules))
    return sudoku_rules, assigned

In [4]:
import numpy as np
import random

# ---- original repo: https://github.com/marcmelis/dpll-sat/blob/master/solvers/base_sat.py#L105
# Simplify the cnf given a certain variable
def simplify_cnf(cnf, variable):
    new_cnf = []
    for clause in cnf:
        # If the variable is in the clause, then remove the clause from the cnf.
        if variable in clause:
            continue
        # If the negated variable is in the clause, then remove the negated variable from the clause
        if -variable in clause:
            new_clause = [item for item in clause if item != -variable]
            new_cnf.append(new_clause)
        else:
            new_cnf.append(clause)
    return new_cnf

# -------- Unit propagation
def propagate_unit(cnf):
    assigned = []
    unit_clauses = [clause for clause in cnf if len(clause) == 1]
    while unit_clauses:
        unit = unit_clauses[0]
        
        cnf = simplify_cnf(cnf, unit[0])
        
        assigned += [unit[0]]
        unit_clauses = [clause for clause in cnf if len(clause) == 1]
    return cnf, assigned

# -------- Pure literals
def get_all_vars(cnf):
    allVars = []
    for clause in cnf:
        allVars.extend(clause)
    allVars = np.sort(np.unique(allVars))
    return allVars

# This function removes all of the pure literals in the cnf
def propagate_pure(cnf):
    allVars = get_all_vars(cnf)
    pures = allVars[[-x not in allVars for x in allVars]]

    for pure in pures:
        formula = simplify_cnf(cnf, pure)

    return cnf, list(pures)

# This function checks if there is a tautology in the cnf
def find_tautology(cnf):
    for clause in cnf:
        for item in clause:
            if -item in clause:
                return True
    return False

# --------------- Recursive algorithm
def dpll(cnf, heuristic, counter, assigned=[]):
    counter += 1
    # Check contradiction/completed sudoku
    if [] in cnf: 
        return [], counter
    
    if not cnf: return assigned, counter

    # Propagate pure literals
    cnf, pure_assigned = propagate_pure(cnf)
    if [] in cnf:
        print('went wrong at propagate_pure(cnf)')
#     formula, pure_assignment = pure_literal(formula)

    # Propagate unit literals
    cnf, unit_assigned = propagate_unit(cnf)
    assigned = assigned + unit_assigned + pure_assigned
    
    # Check contradiction/completed sudoku
    if [] in cnf: 
        #print('why here')
        return [], counter
    if not cnf: return assigned, counter

    # Select variable to expand upon
    variable = heuristic(cnf)
    solution, counter = dpll(simplify_cnf(cnf, variable), heuristic, counter, assigned + [variable])
    if not solution:
        solution, counter = dpll(simplify_cnf(cnf, -variable), heuristic, counter, assigned + [-variable])

    return solution, counter


In [5]:
import random
import numpy as np

# Dynamic Largest Combined Sum
def DLCS(cnf):
    counter = {}
    for clause in cnf:
        for literal in clause:
            if literal > 0:
                if literal in counter:
                    counter[literal] += 1
                else:
                    counter[literal] = 1
            if literal < 0:
                if -literal in counter:
                    counter[-literal] += 1
                else:
                    counter[-literal] = 1
    return max(counter, key=counter.get)

# Dynamic Largest Individual Sum
def DLID(cnf):
    counter = {}
    for clause in cnf:
        for literal in clause:
            if literal in counter:
                counter[literal] += 1
            else:
                counter[literal] = 1
    return max(counter, key=counter.get)

# random heuristic
def random_literal(cnf):
    list_of_literals = []
    for clause in cnf:
        list_of_literals.extend(clause)
    return random.choice(np.unique(list_of_literals))

# MOM's heuristic (Maximum Occurrence's in Clauses of Minimum Size)
def moms_occurences_nonsat(cnf):
    counter = {}
    for clause in cnf:
        if len(clause) > 1: # only non-satisfied clauses
            for literal in clause:
                if literal in counter:
                    counter[literal] += 1
                else:
                    counter[literal] = 1
    return counter

def moms_function(counter, literal, tuning):
  return ((counter[literal] + counter[-literal]) * (2**tuning)) + (counter[literal] * counter[-literal])

def MOMs(cnf, tuning=5):
  moms_counter = {}
  counter = moms_occurences_nonsat(cnf)
  for literal in counter:
      moms_counter[literal] = moms_function(counter, literal, tuning)
  return max(moms_counter, key=moms_counter.get)


In [7]:
def read_txt_line(line):
    idx = 0
    dmacs = []
    for char in line:
        dmacs_line = []
        if char != ".":
            row_nbr = int(idx / 9) + 1 
            column_nbr = idx % 9
            total_dmacs_val = f"{row_nbr}{column_nbr}{char}"
            #print(total_dmacs_val)
            dmacs_line.append(total_dmacs_val)
          #print(dmacs_line)
        #print(dmacs_line)
        if len(dmacs_line) > 0:
           dmacs.extend(dmacs_line)
    return dmacs

In [ ]:
################# RUN EVERYTHING ABOVE THIS AND THIS
#Read in the rules
filename = 'sudoku-rules.txt'
sudoku_rules = open(filename, 'r')
rules = dimac_Reader(sudoku_rules, filename.strip('.txt').replace('-',' ').capitalize(), info = False);
for i, item in enumerate(rules):
    for j, var in enumerate(item):
         rules[i][j] = int(rules[i][j])
            
#Read in sudoku examples
path = 'test sudokus/1000 sudokus.txt'
sudoku1000=open(path,'r')
sudoku_lines = sudoku1000.read().split('\n')

#select heuristic
heuristics = [random_literal, DLCS, DLID, moms_occurences_nonsat, MOMs]

#select dims: #heuristics * # sudokus
number_sudokus = 1
counters = [[0]*number_sudokus]*len(heuristics)
solutions = [['']*number_sudokus]*len(heuristics)

#One by one
for i, heuristic in enumerate(heuristics):
    print(heuristic)
    for j, line in enumerate(sudoku_lines[:number_sudokus]):
        dmacs = read_txt_line(line)
        cnf, assigned = join_input(rules, dmacs)
        #print(cnf)
        heur = heuristic
        solution, counter = dpll(cnf, heur, 0, assigned)
        counters[i][j] = (counter)
        if solution == []:
            solutions[i][j] = 'failed'
        else: solutions[i][j]= 'yay'
print(counters)
print(solutions)

<function random_literal at 0x000001E1485D8D30>
<function DLCS at 0x000001E1485D8C10>


In [18]:
filename = 'sudoku-example.txt'
sudoku = open(filename,'r')
sudoku_input = dimac_Reader(sudoku, filename.strip('.txt').replace('-',' ').capitalize(), info = False, sudoku_vals = True);

filename = 'sudoku-rules.txt'
sudoku_rules = open(filename, 'r')
rules = dimac_Reader(sudoku_rules, filename.strip('.txt').replace('-',' ').capitalize(), info = False);
for i, item in enumerate(rules):
    for j, var in enumerate(item):
         rules[i][j] = int(rules[i][j])

for item in sudoku_input:
    item = int(item)
    assert type(item) == int
print( sudoku_input)
for i, item in enumerate(sudoku_input):
    if ' ' in item: sudoku_input[i] = item.replace(' ', '')
#print(sudoku_input)


['168 ', '175 ', '225 ', '231 ', '318 ', '419 ', '444 ', '465 ', '493 ', '689 ', '692 ', '727 ', '732 ', '828 ', '886 ', '956 ', '961 ', '973 ']


NameError: name 'sudoku_reader' is not defined

In [36]:
def sudoku_reader(path): #'/content/1000 sudokus.txt'
    sudoku1000=open(path,'r')
    sudoku_lines = sudoku1000.read().split('\n')
    dmacs = []
    for line in sudoku_lines:
      idx = 0
      for char in line:
        dmacs_line = []
        if char != ".":
          row_nbr = int(idx / 9) + 1 
          column_nbr = idx % 9
          total_dmacs_val = f"{row_nbr}{column_nbr}{char}"
          #print(total_dmacs_val)
          dmacs_line.append(total_dmacs_val)
          #print(dmacs_line)
        #print(dmacs_line)
        if len(dmacs_line) > 0:
           dmacs.extend(dmacs_line)
        idx += 1
      break
    return dmacs

In [30]:
dmacs = sudoku_reader('test sudokus/1000 sudokus.txt')


119
['119']
124
['124']
161
['161']
173
['173']
347
['347']
356
['356']
382
['382']
418
['418']
441
['441']
513
['513']
522
['522']
632
['632']
676
['676']
745
['745']
764
['764']
858
['858']
887
['887']
926
['926']
933
['933']
954
['954']
988
['988']


['119',
 '124',
 '161',
 '173',
 '347',
 '356',
 '382',
 '418',
 '441',
 '513',
 '522',
 '632',
 '676',
 '745',
 '764',
 '858',
 '887',
 '926',
 '933',
 '954',
 '988']